<a href="https://colab.research.google.com/github/anthonyhu25/Variance-Reduction-Metropolis/blob/main/variance_reduction_for_metropolis_hastings_example_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import random
from numpy import linalg
import math
import scipy
import scipy.stats
import matplotlib.pyplot as plt
from scipy.stats import rv_continuous, rv_discrete
from scipy.stats._distn_infrastructure import rv_frozen
from scipy.special import logsumexp
import warnings
import sys
import statistics
import pandas as pd
from IPython.display import display, Math, HTML

This notebook, as well as the code in the other notebooks in this directory, will come from [this paper](https://arxiv.org/pdf/2203.02268).

# Example 3.1: Simulated Data Example: Gaussian Target


There are a couple of things I must note about the setup to this problem:
1. The coefficient-less estimator of $F: \mu_{n,G}(F):= \frac{1}{n}\sum_{i=0}^{n-1}[F(x_{i}) + \int α(x_{i}, y)(G(y) - G(x))q(y|x_{i})dy]$ needs a specified function $G(x)$ and also analytically evaluate the integral inside the estimator.

To first get an estimate $G$ to approximate $F$ (which can be estimated by expectation of $F$ with respect to target distribution $\pi(F)$), we need a Gaussian approximation of $\pi(x)$ first. We hope that $F_{\pi ̃}$ is a good approximation of the ideal function $F ^{ ̃}$, which is also an estimate of $F$. For this estimation, we set $G$

To estimate the integral $∫ \alpha(x_{i}, y)(G(y) - G(x))q(y|x_{i})dy$, we use Monte-Carlo estimates $α(x_{i}, y_{i})(G(y_{i}) - G(x_{i})), y_{i} \sim q(y|x_{i})$. To further reduce the variance of this estimator (since the Monte-Carlo estimates of the integral can have a high variance) we add in control variate $h(x_{i}, y)$ and $E_{q(y|x_{i})}(h(x_{i},y))$. Note that these terms $E_{q(y|x_{i})}(h(x_{i},y))$ and $h(x_{i},y)$ are static control variates, and also depends on the Gaussian approximation of $\pi(x)$.

So, to estimate the coefficient-less estimator of $F$ above, we use Monte-Carlo methods and use:

$\large \mu_{n, G}(F) := \frac{1}{n}\sum_{i=0}^{n-1}[F(x_{i}) + \alpha(x_{i}, y_{i})(G(y_{i}) - G(x_{i})) + h(x_{i}, y_{i}) - E_{q(y|x_{i})}[h(x_{i}, y)]]$

2. To obtain our static control variate $h(x_{i}, y)$, we first need Gaussian approximations of our target $\pi(x)$ and proposal $q(y|x)$ - let us name them $\pi^{̃}(x)$ and $q^{̃}(y|x)$ respectively - and the function $G(x)$. Then, we set $h(x,y)$ to be the product of the Metropolis-Hastings acceptance ratio between $\pi^{̃}(x)$ and $q^{̃}(y|x)$, and the difference between $G(y)$ and $G(x)$. Formally,

$\large h(x,y) = min(1, r^{̃}(x,y))[G(y)-G(x)]$

where $r^{̃}(x,y) = \frac{\pi^{̃}(y)q^{̃}(x|y)}{\pi^{̃}(x)q^{̃}(y|x)}$

We hope that the acceptance ratio of the Gaussian approximations also approximates the true acceptance ratio between the proposal and the density distributions.

Back to the beginning...

The paper begins by assuming a Markov transition kernel $P$ invariant to a target $\pi$ (if the Markov Chain transition kernel is defined as $P$ and if the current state is distributed as some distribution $\pi$, then after one step the current state is still distributed as $\pi$), a function $G(x)$, and conditional next-step expectation of $G(x)$ with respect to transition kernel $P$ as $PG(x)$, given current state $x$.

We can represent the conditional expectation $PG(x)$ as:

$\large PG(x) := \int P(x, dy)G(y) = G(x) + \int \alpha(x,y)(G(y) - G(x))q(y|x)dy$

where $\alpha(x,y)  = min(1, r(x,y))$, and $r(x,y) := \frac{\pi(y)q(x|y)}{\pi(x)q(y|x)}$

Suppose we have $n$ correlated samples from target density $\pi$. The estimator $\mu_{n,G}$ is unbiased:

$\large \mu_{n,G}(F) = \frac{1}{n}\sum_{i=1}^{n}[F(x_{i}) + PG(x_{i}) - G(x_{i})]$

We substitute $PG(x)$ into $\mu_{n,G}(F)$ and obtain:

$\large \mu_{n,G}(F) = \frac{1}{n}\sum_{i=1}^{n}[F(x_{i}) + \int \alpha(x_{i},y)(G(y)-G(x))q(y|x_{i})dy]$

Then, we approximate the integral $\int \alpha(x_{i},y)(G(y)-G(x))q(y|x_{i})dy$ using a single-sample Monte-Carlo estimate $\alpha(x_{i},y_{i})(G(y_{i}) - G(x_{i})), y_{i} \sim q(y|x_{i})$.

Also, we seek to reduce the variance of the unbiased estimator $\alpha(x_{i},y_{i})(G(y_{i}) - G(x_{i}))$ by adding in a static control variate terms $h(x_{i}, y_{i})$ and $𝔼_{q(y|x_{i})}[h(x_{i},y)]$, which both depends on the Gaussian approximation $\pi^{̃}(x) = N(x|\mu, \Sigma)$ of the target distribution $\pi(x)$.

So, the final estimator $\mu_{n,G}(F)$ becomes:

$\large \mu_{n,G}(F) = \frac{1}{n}\sum_{i=1}^{n}[F(x_{i}) + \alpha(x_{i},y_{i})(G(y_{i})-G(x_{i})) + h(x_{i}, y_{i}) - 𝔼_{q(y|x_{i})}{h(x_{i},y)}]$

## How to construct the static control variates

So we need to construct $h(x_{i}, y_{i})$ and $𝔼_{q(y|x_{i})}{h(x_{i},y)}$ from the Gaussian approximation of the target density $\pi^{̃}(x) \sim N(x|\mu \Sigma)$. Note that $h(x_{i}, y_{i})$ is similar to $α(x_{i}, y_{i})$ in that it is the acceptance ratio between the target $π^{̃}(x)$ and corresponding proposal $q^{̃}(x)$ multiplied by the difference in function G. Formally, $h(x,y) = min(1, r^{̃}(x,y))[G(y)-G(x)]$.

To construct our other static control variate $𝔼_{q(y|x)}{h(x,y)}$, we note that we can reuse the construction of the original $PG(x)$ [here](https://arxiv.org/pdf/2203.02268#page=5)...

$\large 𝔼_{q(y|x)}{h(x,y)} = \int h(x,y)q(y|x)dy = \int min(1, r^{̃}(x,y))[G(y)-G(x)]q(y|x)dy$

Note that we stated that $G(x) = G_{0}(L^{-1}(x-\mu))$, so we substitute this identity back into the above equation. In essense, this is performing the "change of variables" transformation when going from the target/proposal to the Gaussian approximation of the target/proposal.

$\large = \int min(1, r^{̃}(x,y))[G_{0}(L^{-1}(y-\mu))-G_{0}(L^{-1}(x-\mu))]q(y|x)dy$

Since $r^{̃}(x,y)$

The target distribution of this example is a d-variate standard Gaussian distribution $N(0_{d}, I_{d})$ with a proposal distribution $q(y|x) ∼ N(y|x, c^{2}I_{d})$ where $c^{2} = 2.38^{2}/d$ for the Random Metropolis Walk case. We are interested in estimating the expected value of the first coordinate of the target, so $F(x) = x^{(1)}$

To begin, we need to construct our function $G(x)$ and its conditional expectation $PG(x) = 𝔼_{x}(G(X_{1})) = 𝔼_{x}[G(X_{1})|X_{0} = x]$. Note that we are given a $G_{0}(x)$ function, and transform it back to $G(x) = G_{0}(L^{-1}(x - \mu))$, where $L$ is the Cholesky factor for the Gaussian approximation of the target distribution $π(x)$, and $\mu$ is the mean of the Gaussian approximation of the target $\pi(x)$ -- we call this approximation $\pi^{̃}(x) \sim N(x|\mu, \Sigma)$.

Since the target $N(0_{d}, I_{d})$ is already a standard Gaussian distribution, the $G(x)$ in this problem equals $G_{0}(x)$, which is defined as:

$\large G_{0}(x) = b_{0}(e^{b_{1}x^{(j)}} - e^{-b_{1}x^{(j)}}) * e^{-b_{2}||x||^{2}} +
c_{0}(e^{-c_{1}(x^{(j)} - c_{2})^{2}} - e^{-c_{1}(x^{(j)} + c_{2})^{2}}) * e^{-c_{1} \sum_{j^{`} \neq j }(x^{(j^{`})})^{2}}$

Note that $j$ is the coordinate we are trying to estimate from $F(x) = x^{(j)}$, so in this case $j$ equals 1. Also, $b_{0}, b_{1}, b_{2}, c_{0}, c_{1}, c_{2}$ are parameters used for the closed-form approximation of $\alpha_{g}(x)$

In [2]:
# G_0_x function given in paper
def G_0_x(dict_params, x, coordinate):
  arg_1 = dict_params['b0'] *\
   (math.exp(dict_params['b1'] * x[coordinate - 1]) -\
    math.exp(-1 * dict_params['b1'] * x[coordinate - 1]))*\
    math.exp(dict_params['b2'] * (np.linalg.norm(x)** 2))
  arg_2 = dict_params['c0'] * \
   (math.exp(-1 * dict_params['c1'] * (x[coordinate - 1] - dict_params['c2']**2)) -\
    math.exp(-1 * dict_params['c1'] * (x[coordinate - 1] - dict_params['c2'])** 2)) *\
    math.exp(-dict_params['c1'] * (float(np.linalg.norm(np.delete(x, coordinate))) ** 2))
  return arg_1 + arg_2

The pdf of a noncentral Chi-squared distribution $p(f)$ can be represented by modified Bessel function of the first kind $I_{v}(c)$, where $v$ is the degrees of freedom specified. Note that there is another representation of the pdf as an infinite sum of mixtures of Poisson and Gamma (or Poisson and Chi-squared)  distributions, but to analytically evaluate this integral we will use the modified Bessel function of the first kind

So, $p(f)$ can be represented below as:

$\large p(f|k, λ) = \frac{1}{2}e^{-(x+\lambda)/2} (\frac{x}{λ})^{\frac{k}{4} - \frac{1}{2}} I_{k/2-1}(\sqrt{λx})$

where $k$ is degrees of freedom, $\lambda$ is noncentrality parameter, and $I_{k/2-1}(\sqrt{λx})$ is the modified Bessel function of the first form, or:

$\large I_{b}(a) = (a/2)^{b} \sum_{i=0}^{∞} \frac{ (a^{2}/4)^{i}}{i! *Γ(b + i + 1)}$

We need this for an analytically solving $a(x)$ and $a_{g}(x)$ for the



In [3]:
# Integral of the expected min of 1, and ....
## Used for construction of a(x) and a_g(x)
### x.. vector
### d = degrees of freedom, and also in this case, dimension of MVN variables
### c = step size parameter
### is_MALA = boolean parameter, needed for calculation of tau
def chi_squared_expectation(x, d, c, non_central_param, is_MALA):
  norm_x = np.linalg.norm(x)
  # calculate bounds of boundary
  boundary_change = (norm_x/c) ** 2
  if is_MALA == True:
    tau = c/2
  else:
    tau = 1
  # PDF of noncentral chi-squared in terms of modified bessel function
  ## f is vector
  ## d is degrees of freedom
  ### v is non-centrality parameter
  def pdf_chi_squared(f, d, v):
    return 0.5 * np.exp(-(f + v)/2) * ((f/v) ** (d / 4 - 1/2)) * scipy.special.iv(d / 2 -1, math.sqrt(v * f))
  # Calculate integral "left" of boundary
  ## in terms of f, because f is the non-central chi-squared distribution
  ### product integrated over from 0 to boundary change
  ### Function: pdf of chi-squared multiplied by some exponential term
  left_integral = scipy.integrate.quad(lambda f: pdf_chi_squared(f, d, non_central_param) *\
                                       np.exp(-( (c * tau) ** 2 / 2) * (f - boundary_change)), 0, boundary_change)
  # Calculate integral "rigtht" of boundary
  ## much simpler: only in terms of chi-squared pdf
  right_integral = scipy.integrate.quad(lambda f: pdf_chi_squared(f, d, non_central_param),
                                        boundary_change, np.inf)
  return left_integral[0] + right_integral[0]

I do notice some mistakes in the paper. The integral of the min function with respect to the noncentral chi-square distributions appear to have the wrong bounds -- I believe they should be swapped.

Also, there is inconsistent naming of the noncentral parameter for $a(x)$

In [4]:
# calculation of a_g_x
## Note that this is basically a weighted mixtured of the sum of K chi-squared min expectations
## x: vector
## K: number of components used
## c: stepsize, also used for calculation of r
## d: dimension of sample/vector x
## beta_list: list of vectors b1,...bK
## rho_list: list of scalars rho1,...rhoK
## delta_list: list of vectors delta1,...deltaK
## is_MALA: boolean based on whether algorithm is MALA or RWM(true if MALA, false otherwise)


def a_g_x_calculation(x, K, c, d, beta_list, rho_list, delta_list, is_MALA):
  r = 1
  if is_MALA == True:
    r = 1 - (c ** 2)/2
  sum = 0
  for i in range(K):
    # Calculate m_k(x)
    m_k_x = (r * x + (c ** 2) * (beta_list[i] + rho_list[i] * delta_list[i])) / (1 + 2 * rho_list[i] * (c ** 2))
    # Calculate A_k(x)
    A_k = ((1 + 2 * rho_list[i] * (c ** 2)) ** (-d/2)) *\
     np.exp( (-1/2 * (r * np.linalg.norm(x)/c) ** 2) -\
    (rho_list[i] * np.linalg.norm(delta_list[i]) ** 2) +\
    (np.linalg.norm(m_k_x) ** 2)/(2 * (c ** 2) * (1 + 2 * rho_list[i] * (c ** 2))))
    # Find the "c" parameter for calculating expectation
    ## It's called s_k2 for each k = 1,...K
    s_k2 =  np.sqrt( (c ** 2)/ (1 + 2 * (c ** 2) * rho_list[i]))
    ## calculate noncentral parameter for k
    noncentral_param_k = (np.linalg.norm(m_k_x)/c)**2
    sum += A_k * chi_squared_expectation(x, d, c, noncentral_param_k, is_MALA)
  return sum

In [5]:
# Dictionary of parameters
## Given in example for RWM

rwm_params = dict(
    b0=8.7078,
    b1=0.2916,
    b2=0.0001,
    c0=-3.5619,
    c1=0.1131,
    c2=3.9162
)


In [31]:
def metropolis_hastings_example_3_1(d, params, is_MALA, n_burn_in, n_samples, T_iterations):
  # Initiate mu_lists: will append each 1:T_iterations
  mu_MC = []
  mu_CV = []
  mu_CV_coeff = []
  acceptance_counter_list = []
  target_density = scipy.stats.multivariate_normal(mean = [0 for _ in range(d)], cov = np.eye(d))
  # Calculate c and r parameters before we initiate chain
  ## r depends on if we are doing MALA or RWM
  c_squared = (2.38 ** 2)/d
  r = 1
  for i in range(T_iterations):
    # X Chain is state of chain
    # Y Chain is proposal state of chain
    alpha_chain = []
    X_chain = []
    Y_chain = []
    acceptance_counter = 0
    # Initiate proposal distribution
    proposal_density = lambda z : scipy.stats.multivariate_normal(mean = z, cov = c_squared * np.eye(d))
    for j in range(n_samples + n_burn_in):
      if j == 0:
        # Accept with probability 1
        Y_j = proposal_density(z = [0 for _ in range(d)]).rvs()
        alpha_j = 1
        alpha_chain.append(alpha_j)
        Y_chain.append(Y_j.astype(float))
        X_chain.append(Y_j.astype(float))
      else:
        # Get current state
        x = X_chain[len(X_chain)-1]
        y = proposal_density(z = r * x).rvs()
        alpha_j = min(1, (proposal_density(z = r * y).pdf(x) * target_density.pdf(y))/(proposal_density(z = r * x).pdf(y) * target_density.pdf(x)))
        # Generate uniform for acceptance/rejection
        U = scipy.stats.uniform(0, 1).rvs()
        if U <= alpha_j:
          # Cannot throw out burn-in samples for X and Y chains yet. need them for the others,
          # but for alpha chains and acceptance we can start counting once burn-in period stops
          Y_chain.append(y.astype(float))
          X_chain.append(y.astype(float))
          if j >= n_burn_in:
            alpha_chain.append(float(alpha_j))
            acceptance_counter += 1
        else:
          Y_chain.append(y.astype(float))
          X_chain.append(x.astype(float))
          if j >= n_burn_in:
            alpha_chain.append(float(alpha_j))
    # After chain is simulated, drop burn-in samples
    alpha_chain = alpha_chain[:n_burn_in]
    X_chain = X_chain[:n_burn_in]
    Y_chain = Y_chain[:n_burn_in]
    # Calculate means
    ## Base MC method
    mu_MC_i = np.mean([X_chain[j][0] for j in range(len(X_chain))])
    mu_MC.append(float(mu_MC_i))
    ## Control variate
    ### need G function and approximation of Gaussian
    ### but since both distributions (target and proposal) are Gaussian, do not need to re-simulate/redraw, instead, calculate using G function
    ### Also note that since the target is a standard normal, no transformation (for covariance and mean/shift) is needed for target
    #### Static control variate term
    h_list = [alpha_chain[i] * (G_0_x(params, X_chain[i], 1) - G_0_x(params, Y_chain[i], 1)) for i in range(len(X_chain))]
    #### expecatation of h(x,y) wrt. proposal density
    expectation_h_list = []

  return mu_MC





In [34]:
test_run = metropolis_hastings_example_3_1(2, rwm_params, False, 1000, 1000, 10)

In [35]:
print(test_run)

[-0.17081496481686526, -0.20454703209667127, 0.16439307839653378, -0.09199691854861056, 0.05884479445594152, -0.10575516800035115, 0.021664328514220045, 0.002757346494075698, 0.0316617068796241, 0.1699133133081956]
